# Relay PrintIR

In [1]:
import tvm
from tvm import relay

shape = (1, 2, 3)
tp = relay.TensorType(shape, "float32")
x = relay.var("x", tp)
y = relay.add(x, x)
y = relay.multiply(y, relay.const(2, "float32"))
func = relay.Function([x], y)

seq = tvm.transform.Sequential(
    [
        relay.transform.InferType(),
        relay.transform.FoldConstant(),
        tvm.transform.PrintIR(),
        relay.transform.DeadCodeElimination(),
    ]
)

mod = tvm.IRModule({"main": func})
with tvm.transform.PassContext(opt_level=3):
    mod = seq(mod)

[21:04:47] /workspace/tvm/src/ir/transform.cc:655: PrintIR():
#[version = "0.0.5"]
def @main(%x: Tensor[(1, 2, 3), float32] /* ty=Tensor[(1, 2, 3), float32] */) -> Tensor[(1, 2, 3), float32] {
  %0 = add(%x, %x) /* ty=Tensor[(1, 2, 3), float32] */;
  multiply(%0, 2f /* ty=float32 */) /* ty=Tensor[(1, 2, 3), float32] */
}



In [ ]:
out = capfd.readouterr().err

assert "PrintIR" in out
assert "multiply" in out